# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import desc, asc, sum as Fsum


import datetime

import numpy as np
import pandas as pd
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [3]:
spark = SparkSession.builder.appName("Spark SQL").getOrCreate()
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.createOrReplaceTempView("user_logs")

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id ""(empty string) NOT visit?

In [9]:
spark.sql("""
        SELECT DISTINCT page
        FROM user_logs
        WHERE userId = ''
""").show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [12]:
spark.sql("""
        SELECT COUNT(DISTINCT(userId))
        FROM user_logs
        WHERE gender = 'F'
""").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [22]:
spark.sql("""
        SELECT MAX(song_count) FROM
        (SELECT COUNT(song) as song_count
        FROM user_logs
        GROUP BY artist)
""").show()

+---------------+
|max(song_count)|
+---------------+
|             83|
+---------------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [23]:
is_home = spark.sql("""
        SELECT userId, page, ts,
        CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home
        FROM user_logs
        WHERE page = 'Home' OR page = 'NextSong'
""")

In [25]:
is_home.createOrReplaceTempView('is_home_view')

In [36]:
spark.sql("""
        SELECT AVG(count) FROM
        (SELECT COUNT(*) as count FROM
        (SELECT *, SUM(is_home) OVER
        (PARTITION BY userId ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
        FROM is_home_view)
        GROUP BY userId, period, page
        HAVING page = 'NextSong')
""").show()

+-----------------+
|       avg(count)|
+-----------------+
|6.898347107438017|
+-----------------+

